<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%204/4_6_Bilderkennung_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import img_to_array, array_to_img
from keras.preprocessing.image import img_to_array, array_to_img

from keras.utils.vis_utils import plot_model

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras import layers, models

warnings.simplefilter('always', category=UserWarning)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import rcParams
import string
rcParams['figure.figsize'] = 14, 10

In [ ]:
def show_accuracy_loss_plot(history, num_epochs):
    epochs = [i for i in range(num_epochs)]
    fig , ax = plt.subplots(1,2)
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    fig.set_size_inches(16,9)
    
    ax[0].plot(epochs , train_acc , 'go-' , label = 'Train Accuracy')
    ax[0].plot(epochs , val_acc , 'ro-' , label = 'Test Accuracy')
    ax[0].set_title('Training & Test Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")
    
    ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
    ax[1].plot(epochs , val_loss , 'r-o' , label = 'Testing Loss')
    ax[1].set_title('Training & Test Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    plt.show()

# 4.6 Bilderkennung 2

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_sign_language.jpg">



# Einlesen der Daten

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d datamunge/sign-language-mnist

In [ ]:
import zipfile
with zipfile.ZipFile("sign-language-mnist.zip", 'r') as zip_ref:
    zip_ref.extractall("")

In [ ]:
df_train = pd.read_csv("sign_mnist_train.csv")
df_test = pd.read_csv("sign_mnist_test.csv")

In [ ]:
y_test = df_test["label"]
y_train = df_train["label"]
del df_train['label']
del df_test['label']

In [ ]:
# Normalisierung
X_train = df_train.values/255
X_test = df_test.values/255

# Reshaping von 1D zu 3D
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
alphabetic_label = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

# Label Encoding

In [ ]:
lb=LabelBinarizer()
lb.fit(y_train)
y_test_oh = lb.transform(y_test)
y_train_oh = lb.transform(y_train)

# Erweiterung des Datensatzes

Ende der vorherigen Einheit haben wir ein "komplexeres CNN" mit mehr Layern für unser Problem verwendet. In dieser Einheit wollen wir noch einen weiteren Ansatz testen, um die Ergebnisse unseres Modells zu verbessern. 

Wir wollen im Folgenden unseren Bilddatensatz erweitern, indem wir kleine Veränderungen an existierenden Bildern vornehmen und diese auch im Training verwenden. Potentiell können wir Bilder verschieben, verzerren, Zoom anwenden oder spiegeln. Auf Spiegelung haben wir in diesem Datensatz bewusst verzichtet. 

In [ ]:
data_augmentation = ImageDataGenerator(
                                  rotation_range = 0,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  shear_range=0,
                                  horizontal_flip=False,
                                  vertical_flip=False)
data_augmentation.fit(X_train)

In [ ]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(5,5),
                 strides=1,padding='same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=2,padding='same'))
model.add(Conv2D(64,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
model.add(Conv2D(32,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
          
model.add(Flatten())

model.add(Dense(units=256,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=24,activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Wir trainieren hier das gleiche Modell wie vorher auch gezeigt. Nur verwenden wir nicht direkt die Trainingsdaten, sondern erweitern diese künstlich vor dem Training. 

In [ ]:
history = model.fit(data_augmentation.flow(X_train,y_train_oh,batch_size=128),
         epochs = 40,
          shuffle=1,
           validation_data=(X_test,y_test_oh),
         )

In [ ]:
show_accuracy_loss_plot(history, num_epochs=40)

Wir sehen hier eine Verbesserung auf bis zu 99% Accuracy. Da die Klassen in unserem Datensatz sehr ausgeglichen sind, ist die Accuracy eine valide Metrik für die Analyse.

# Transfer Learning & Pretrained Computer Vision Models

In dem folgenden Teil dieser Einheit gehen wir auf ein Thema ein, das nicht zwangsläufig für unsere Bilderkennung von Gebärdensprache wichtig ist, dennoch aber sehr spannend ist: Transfer Learning und Pretrained Machine Learning Models.

Transferlernen ist ein Teilgebiet des maschinellen Lernens und der künstlichen Intelligenz, das darauf abzielt, das aus einer Aufgabe (Ausgangsaufgabe) gewonnene Wissen auf eine andere, aber ähnliche Aufgabe (Zielaufgabe) anzuwenden.
Dabei versucht man den Lernvorgangs in einer neuen Aufgabe durch den Transfer von Wissen zu verbessern und zu verschnellern. Transfer Learning kann die Zeit, neue komplexe Netzwerke zu definieren und zu trainieren deutlich reduzieren. 



Für Transfer Learning werden sogenannte pretrained Modelle verwendet. Diese werden meist von Forschungsgruppen oder Tech-Unternehmen erstellt und auf großen Datensätzen trainiert (z.B. ImageNet oder Wikipedia Corpus). Diese Modelle werden anschließend veröffentlicht und für andere EntwicklerInnen zur Verfügung gestellt. 

Transfer Learning ist nicht beschränkt auf den Bereich Computer Vision.
Hier sind einige Beispiele von pretrained Neural Networks für Computer Vision Aufgaben. Diese können unter anderem für Aufgaben wie Image Classification, Neural Style Transfer oder Anomalie Erkennung genutzt werden. 

- VGG16
- VGG19
- Inceptionv3 (GoogLeNet)
- ResNet50
- EfficientNet

In dieser Einheit werden wir ein VGG16 verwenden mit pretrained Parameters basierend auf Training auf ImageNet, einem großen Datensatz mit über 14 Millionen Bildern. Hierfür müssen wir allerdings unsere Bilder auf eine Größe von 48x48 skalieren, da VGG16 diesen Input als Minimalgröße erwartet. 

In [ ]:
X_train = df_train.values
X_test = df_test.values

X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

x_train_t = np.stack([X_train.reshape(X_train.shape[0],28,28)]*3, axis=3).reshape(X_train.shape[0],28,28,3)
x_test_t = np.stack([X_test.reshape(X_test.shape[0],28,28)]*3, axis=3).reshape(X_test.shape[0],28,28,3)
x_train_t.shape, x_test_t.shape


x_train_tt = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_train_t])
x_test_tt = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_test_t])
x_train_tt.shape, x_test_tt.shape

Nun wollen wir das VGG16 Modell einmal laden. Wir verwenden hier die bereits trainierten Parameter auf dem Datensatz "ImageNet". Anschließend definieren wir den Rest des Modelles und fügen noch weitere Dense-Layer hinzu vor dem Output-Layer. 

Hier der Grobaufbau des VGG16 Modells:

<img width=80% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/VGG_2.png">

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
base_model.trainable = False

train_ds = preprocess_input(x_train_tt) 
test_ds = preprocess_input(x_test_tt)

In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(36, activation='relu')
prediction_layer = layers.Dense(24, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])
model.compile(loss = keras.losses.categorical_crossentropy, optimizer="adam",
              metrics=['accuracy'])

Wir wollen uns erneut einmal die Zusammenfassung unseres Modells ausgeben lassen. Hierbei sehen wir, dass das Modell mehr als 14 Millionen Parameter hat. Unser vorheriges Modell hat lediglich 200k Parameter. Das Modell ist durchaus deutlich zu groß für unsere aktuelle Aufgabe. Wie oben allerdings betont, geht es bei diesem Teil der Einheit nicht darum, ein besseres Ergebnis zu erhalten, sondern das Konzept von Transfer Learning zu verdeutlichen. 

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_ds,y_train_oh, batch_size=128,
         epochs = 10,
          shuffle=1,
           validation_data=(test_ds,y_test_oh),
         )

In [ ]:
show_accuracy_loss_plot(history, num_epochs=10)

Das ist das Ende der Einheit zum Thema Erkennung von Gebärdensprache in Bildern. In der folgenden Einheit **4.7 Ergebnis und Auswertung** werden wir uns erneut unser bestes Modell im Detail ansehen und auswerten. Das Transfer Learning Modell werden wir in Einheit 4.7 nicht betrachten.  